In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('train_dist.csv')

In [3]:
col4 = 'Total no of years Experience [before joining Piramal]'
col5 = 'How many Organization that you have worked before joining Piramal Finance ?'  # remove this column not able to make sense of the data here.
col6 = 'Name of your Previous Organization / Company'   # already included in resume and also there are 487 companies here. Too much noise
col7 = 'How many are earning family members ? [Other then yourself]2'
col8 = 'Branch Pincode'
col9 = 'Residential Pincode'

In [4]:
train_df[col4] = train_df[col4].fillna(0)
# train_df[col5] = train_df[col5].fillna(0)
train_df[col6] = train_df[col6].fillna(0)
train_df[col7] = train_df[col7].fillna(1)
train_df[col8] = train_df[col8].fillna(train_df[col9])

In [5]:
number_columns = ['Total no of years Experience [before joining Piramal]', 'How many are earning family members ? [Other then yourself]2']

In [6]:
outliers_dict = {}
for column_name in number_columns:
    iqr = train_df[column_name].quantile(0.75) - train_df[column_name].quantile(0.25)
    outliers = [value for value in train_df[(train_df[column_name] < train_df[column_name].quantile(0.25) - 1.5 * iqr) | (train_df[column_name] > train_df[column_name].quantile(0.95) + 1.5 * iqr)][column_name]]
    outliers_dict[column_name] = outliers
outliers_dict
col1 = 'Total no of years Experience [before joining Piramal]'
col2 = 'How many are earning family members ? [Other then yourself]2'
train_df.loc[train_df[col1] == outliers_dict[col1][1], col1] = 4
for i in range(len(outliers_dict[col2])):
    train_df.loc[train_df[col2] == outliers_dict[col2][i], col2] = 1


In [7]:
columns_to_drop = ['Name of your Previous Organization / Company', 'How many Organization that you have worked before joining Piramal Finance ?',
                  'CandidateID', 'Designation', 'DOJ', 'Location Code', 'branch_lat', 'branch_lon', 'residential_lat', 'residential_lon', 'train_vector']

In [8]:
columns_to_one_hot = ['Previous Industry worked with [before joining Piramal]', 'Average Incentive [per month] earned in your pervious company ?',
                     'How did you come to know about the role at Piramal Finance ?', 'Which Products you are selling in your pervious role ?',
                     'What was the average ticket size handled at your end in previous role ?', 'How many members are there in your family ?',
                     'Department']

In [9]:
train_df = train_df.drop(columns=columns_to_drop)

In [10]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)  # Set sparse=False for dense output
encoded_data = encoder.fit_transform(train_df[['Previous Industry worked with [before joining Piramal]']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out())


In [11]:
unique = train_df.nunique()
unique

Total no of years Experience [before joining Piramal]                       81
Previous Industry worked with [before joining Piramal]                       6
Average Incentive [per month] earned in your pervious company ?              5
How did you come to know about the role at Piramal Finance ?                 8
Which Products you are selling in your pervious role ?                      58
What was the average ticket size handled at your end in previous role ?      8
How many members are there in your family ?                                  4
How many are earning family members ? [Other then yourself]2                 7
How many members are dependent on you ?                                     10
Department                                                                   8
Residential Pincode                                                        621
Branch Pincode                                                             259
Performance                                         

In [12]:
train_df = train_df.drop(['Previous Industry worked with [before joining Piramal]'], axis=1)

In [ ]:
train_df = train_df.drop(['products'], axis=1)

In [14]:
train_df = pd.concat([train_df, encoded_df], axis=1)

In [15]:
from sklearn.preprocessing import LabelEncoder

# Assuming your DataFrame is called 'df'
df = train_df.copy()

# Split the cell values into separate product categories
df['products'] = df['Which Products you are selling in your pervious role ?'].apply(lambda x: str(x).split(';') if isinstance(x, (str, float)) else [])

# Create a unique set of all the product categories
all_products = set([product for products in df['products'] for product in products])

# Assign a numerical label to each unique product category
label_encoder = LabelEncoder()
label_encoder.fit(list(all_products))
df['Which Products you are selling in your pervious role ?'] = df['products'].apply(lambda x: [label_encoder.transform([p])[0] for p in x])

# Explode the 'Which Products you are selling in your pervious role ?' column to create one row per product
df = df.explode('Which Products you are selling in your pervious role ?')

In [16]:
train_df = train_df.drop(['Which Products you are selling in your pervious role ?'], axis=1)

In [17]:
train_df = pd.concat([train_df, df['Which Products you are selling in your pervious role ?']], axis=1)

In [18]:
from sklearn.preprocessing import LabelEncoder

df1 = train_df.copy()
# Create a unique set of all the sources
all_sources = train_df['How did you come to know about the role at Piramal Finance ?'].unique()

# Assign a numerical label to each unique source
label_encoder = LabelEncoder()
label_encoder.fit(all_sources)
df1['How did you come to know about the role at Piramal Finance ?'] = label_encoder.transform(df1['How did you come to know about the role at Piramal Finance ?'])

In [46]:
from sklearn.preprocessing import LabelEncoder

df2 = train_df.copy()
# Create a unique set of all the sources
all_sources = train_df['education'].unique()

# Assign a numerical label to each unique source
label_encoder = LabelEncoder()
label_encoder.fit(all_sources)
df2['education'] = label_encoder.transform(df1['education'])

In [47]:
train_df = train_df.drop(['education'], axis=1)
train_df = pd.concat([train_df, df2['education']], axis=1)

In [21]:
train_df = train_df.drop(['How did you come to know about the role at Piramal Finance ?'], axis=1)

In [22]:
train_df = pd.concat([train_df, df1['How did you come to know about the role at Piramal Finance ?']], axis=1)

In [103]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Assuming your DataFrame is called 'df'
df = train_df.copy()

# One-hot encode the 'Average Incentive [per month] earned in your pervious company ?'
encoder1 = OneHotEncoder(sparse_output=False)
encoded_data1 = encoder1.fit_transform(df[['Average Incentive [per month] earned in your pervious company ?']])
feature_names_1 = {f'avg_incentive_{i}': i for i in range(encoded_data1.shape[1])}
encoded_df1 = pd.DataFrame(encoded_data1, columns=feature_names_1.keys())

# One-hot encode the 'What was the average ticket size handled at your end in previous role ?'
encoder2 = OneHotEncoder(sparse_output=False)
encoded_data2 = encoder2.fit_transform(df[['What was the average ticket size handled at your end in previous role ?']])
feature_names_2 = {f'ticket_size_{i}': i for i in range(encoded_data2.shape[1])}
encoded_df2 = pd.DataFrame(encoded_data2, columns=feature_names_2.keys())

# One-hot encode the 'How many members are there in your family ?'
encoder3 = OneHotEncoder(sparse_output=False)
encoded_data3 = encoder3.fit_transform(df[['How many members are there in your family ?']])
feature_names_3 = {f'family_members_{i}': i for i in range(encoded_data3.shape[1])}
encoded_df3 = pd.DataFrame(encoded_data3, columns=feature_names_3.keys())

# One-hot encode the 'Department'
encoder4 = OneHotEncoder(sparse_output=False)
encoded_data4 = encoder4.fit_transform(df[['Department']])
feature_names_4 = {f'department_{i}': i for i in range(encoded_data4.shape[1])}
encoded_df4 = pd.DataFrame(encoded_data4, columns=feature_names_4.keys())

put = [train_df.reset_index(drop=True), 
      encoded_df1.reset_index(drop=True),
      encoded_df2.reset_index(drop=True),
      encoded_df3.reset_index(drop=True),
      encoded_df4.reset_index(drop=True)]
# Concatenate the original DataFrame with the encoded DataFrames
train_df = pd.concat(put, axis=1)

# The feature names are now stored in the following dictionaries:
# feature_names_1, feature_names_2, feature_names_3, feature_names_4

KeyError: "None of [Index(['Average Incentive [per month] earned in your pervious company ?'], dtype='object')] are in the [columns]"

In [25]:
drop = ['Average Incentive [per month] earned in your pervious company ?'
       ,'Department']
put = [train_df, encoded_df1, encoded_df2, encoded_df3, encoded_df4]

In [26]:
train_df = train_df.drop(['What was the average ticket size handled at your end in previous role ?'], axis=1)

In [37]:
train_df = train_df.drop(['Average Incentive [per month] earned in your pervious company ?'], axis=1)

In [38]:
train_df = train_df.drop(['Department'], axis=1)

In [27]:
train_df = train_df.drop(['How many members are there in your family ?'], axis=1)

In [28]:
train_df = train_df.drop(['Residential Pincode', 'Branch Pincode'], axis=1)

In [29]:
# put = [train_df.reset_index(drop=True), 
#       encoded_df1.reset_index(drop=True),
#       encoded_df2.reset_index(drop=True),
#       encoded_df3.reset_index(drop=True),
#       encoded_df4.reset_index(drop=True)]

# train_df = pd.concat(put, axis=1)

In [39]:
pre = pd.read_csv('train_dist.csv')
pre.dtypes

CandidateID                                                                     object
Designation                                                                     object
Total no of years Experience [before joining Piramal]                          float64
Previous Industry worked with [before joining Piramal]                          object
Name of your Previous Organization / Company                                    object
How many Organization that you have worked before joining Piramal Finance ?     object
Average Incentive [per month] earned in your pervious company ?                 object
How did you come to know about the role at Piramal Finance ?                    object
Which Products you are selling in your pervious role ?                          object
What was the average ticket size handled at your end in previous role ?         object
How many members are there in your family ?                                     object
How many are earning family members ? [Othe

In [48]:
pd.set_option('display.max_columns', None)
train_df

,Total no of years Experience [before joining Piramal],How many are earning family members ? [Other then yourself]2,How many members are dependent on you ?,Performance,distance,Previous Industry worked with [before joining Piramal]_Banking,Previous Industry worked with [before joining Piramal]_Insurance,Previous Industry worked with [before joining Piramal]_NBFC,Previous Industry worked with [before joining Piramal]_Non NBFC,Previous Industry worked with [before joining Piramal]_Others,Previous Industry worked with [before joining Piramal]_others,Which Products you are selling in your pervious role ?,How did you come to know about the role at Piramal Finance ?,avg_incentive_0,avg_incentive_1,avg_incentive_2,avg_incentive_3,avg_incentive_4,ticket_size_0,ticket_size_1,ticket_size_2,ticket_size_3,ticket_size_4,ticket_size_5,ticket_size_6,ticket_size_7,family_members_0,family_members_1,family_members_2,family_members_3,department_0,department_1,department_2,department_3,department_4,department_5,department_6,department_7,education
0,5.0,1.0,1,0,28.003448,0.0,0.0,1.0,0.0,0.0,0.0,9,5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
1,5.0,1.0,1,0,28.003448,0.0,0.0,1.0,0.0,0.0,0.0,10,5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
2,5.0,1.0,1,0,28.003448,0.0,0.0,1.0,0.0,0.0,0.0,0,5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
3,1.5,2.0,4,1,16665.390393,1.0,0.0,0.0,0.0,0.0,0.0,12,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
4,1.5,2.0,4,1,16665.390393,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,5.0,1.0,2,1,295.012198,1.0,0.0,0.0,0.0,0.0,0.0,9,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1620,5.0,1.0,2,1,295.012198,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1621,5.0,1.0,3,1,263.838903,0.0,0.0,1.0,0.0,0.0,0.0,1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3
1622,5.0,1.0,3,1,263.838903,0.0,0.0,1.0,0.0,0.0,0.0,10,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3


In [49]:
train_df.to_csv('final_train.csv')

In [50]:
accuracy = {}
speed = {}

In [51]:
train_df['Which Products you are selling in your pervious role ?'] = train_df['Which Products you are selling in your pervious role ?'].astype(int)

In [54]:
X = train_df.drop('Performance', axis=1)
y = train_df['Performance']

In [57]:
X = X.rename(columns={'Total no of years Experience [before joining Piramal]': 'Total no of years Experience before joining Piramal',
                     'How many are earning family members ? [Other then yourself]2': 'How many are earning family members Other then yourself',
                     'Previous Industry worked with [before joining Piramal]_Banking': 'Previous Industry worked with before joining Piramal_Banking',
                     'Previous Industry worked with [before joining Piramal]_Insurance': 'Previous Industry worked with before joining Piramal_Insurance',
                     'Previous Industry worked with [before joining Piramal]_NBFC':'Previous Industry worked with before joining Piramal_NBFC',
                     'Previous Industry worked with [before joining Piramal]_Non NBFC':'Previous Industry worked with before joining Piramal_Non NBFC',
                     'Previous Industry worked with [before joining Piramal]_Others': 'Previous Industry worked with before joining Piramal_Others',
                     'Previous Industry worked with [before joining Piramal]_others':'Previous Industry worked with before joining Piramal_others'})

In [61]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import numpy as np

model = XGBClassifier()

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=5, random_state=12)
score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)

accuracy['XGBoost'] = (np.mean(score) * 100).round(3)
accuracy['XGBoost'] 

84.983

In [65]:
import joblib

# Train the model
model = XGBClassifier()
model.fit(X, y)

# Save the model
joblib.dump(model, 'xgboost_model.joblib')

['xgboost_model.joblib']

In [62]:
X.columns

Index(['Total no of years Experience before joining Piramal',
       'How many are earning family members Other then yourself',
       'How many members are dependent on you ?', 'distance',
       'Previous Industry worked with before joining Piramal_Banking',
       'Previous Industry worked with before joining Piramal_Insurance',
       'Previous Industry worked with before joining Piramal_NBFC',
       'Previous Industry worked with before joining Piramal_Non NBFC',
       'Previous Industry worked with before joining Piramal_Others',
       'Previous Industry worked with before joining Piramal_others',
       'Which Products you are selling in your pervious role ?',
       'How did you come to know about the role at Piramal Finance ?',
       'avg_incentive_0', 'avg_incentive_1', 'avg_incentive_2',
       'avg_incentive_3', 'avg_incentive_4', 'ticket_size_0', 'ticket_size_1',
       'ticket_size_2', 'ticket_size_3', 'ticket_size_4', 'ticket_size_5',
       'ticket_size_6', 'ticket_

In [193]:
test = pd.read_csv('test_dist.csv')

In [194]:
test.columns

Index(['CandidateID', 'Designation',
       'Total no of years Experience [before joining Piramal]',
       'Previous Industry worked with [before joining Piramal]',
       'Name of your Previous Organization / Company',
       'How many Organization that you have worked before joining Piramal Finance ?',
       'Average Incentive [per month] earned in your pervious company ?',
       'How did you come to know about the role at Piramal Finance ?',
       'Which Products you are selling in your pervious role ?',
       'What was the average ticket size handled at your end in previous role ?',
       'How many members are there in your family ?',
       'How many are earning family members ? [Other then yourself]2',
       'How many members are dependent on you ?', 'Department', 'DOJ',
       'Location Code', 'Residential Pincode', 'Branch Pincode', 'education',
       'train_vector', 'branch_lat', 'branch_lon', 'residential_lat',
       'residential_lon', 'distance'],
      dtype='objec

In [195]:
test[col4] = test[col4].fillna(0)
# train_df[col5] = train_df[col5].fillna(0)
test[col6] = test[col6].fillna(0)
test[col7] = test[col7].fillna(1)
test[col8] = test[col8].fillna(test[col9])

In [196]:
outliers_dict = {}
for column_name in number_columns:
    iqr = test[column_name].quantile(0.75) - test[column_name].quantile(0.25)
    outliers = [value for value in test[(test[column_name] < test[column_name].quantile(0.25) - 1.5 * iqr) | (test[column_name] > test[column_name].quantile(0.95) + 1.5 * iqr)][column_name]]
    outliers_dict[column_name] = outliers
outliers_dict
col1 = 'Total no of years Experience [before joining Piramal]'
col2 = 'How many are earning family members ? [Other then yourself]2'
#test.loc[test[col1] == outliers_dict[col1][1], col1] = 4
for i in range(len(outliers_dict[col2])):
    test.loc[test[col2] == outliers_dict[col2][i], col2] = 1


In [197]:
columns_to_drop = ['Name of your Previous Organization / Company', 'How many Organization that you have worked before joining Piramal Finance ?',
                  'CandidateID', 'Designation', 'DOJ', 'Location Code', 'branch_lat', 'branch_lon', 'residential_lat', 'residential_lon', 'train_vector']

In [198]:
test = test.drop(columns=columns_to_drop, axis=1)

In [199]:
from sklearn.preprocessing import OneHotEncoder

encoder5 = OneHotEncoder(sparse_output=False)  # Set sparse=False for dense output
encoded_data = encoder5.fit_transform(test[['Previous Industry worked with [before joining Piramal]']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder5.get_feature_names_out())


In [200]:
test = test.drop(['Previous Industry worked with [before joining Piramal]'], axis=1)

In [201]:
test = pd.concat([test, encoded_df], axis=1)

In [202]:
from sklearn.preprocessing import LabelEncoder

# Assuming your DataFrame is called 'df'
df3 = test.copy()

# Split the cell values into separate product categories
df3['products'] = df3['Which Products you are selling in your pervious role ?'].apply(lambda x: str(x).split(';') if isinstance(x, (str, float)) else [])

# Create a unique set of all the product categories
all_products = set([product for products in df3['products'] for product in products])

# Assign a numerical label to each unique product category
label_encoder = LabelEncoder()
label_encoder.fit(list(all_products))
df3['Which Products you are selling in your pervious role ?'] = df3['products'].apply(lambda x: [label_encoder.transform([p])[0] for p in x])

# Explode the 'Which Products you are selling in your pervious role ?' column to create one row per product
df3 = df3.explode('Which Products you are selling in your pervious role ?')

In [203]:
test = test.drop(['Which Products you are selling in your pervious role ?'], axis=1)

In [204]:
test = pd.concat([test, df3['Which Products you are selling in your pervious role ?']], axis=1)

In [205]:
from sklearn.preprocessing import LabelEncoder

df4 = test.copy()
# Create a unique set of all the sources
all_sources = df4['How did you come to know about the role at Piramal Finance ?'].unique()

# Assign a numerical label to each unique source
label_encoder = LabelEncoder()
label_encoder.fit(all_sources)
df4['How did you come to know about the role at Piramal Finance ?'] = label_encoder.transform(df4['How did you come to know about the role at Piramal Finance ?'])

In [206]:
from sklearn.preprocessing import LabelEncoder

df5 = test.copy()
# Create a unique set of all the sources
all_sources = df5['education'].unique()

# Assign a numerical label to each unique source
label_encoder = LabelEncoder()
label_encoder.fit(all_sources)
df5['education'] = label_encoder.transform(df5['education'])

In [207]:
test = test.drop(['education'], axis=1)
test = pd.concat([test, df5['education']], axis=1)

In [208]:
test = test.drop(['How did you come to know about the role at Piramal Finance ?'], axis=1)

In [209]:
test = pd.concat([test, df4['How did you come to know about the role at Piramal Finance ?']], axis=1)

In [218]:

from sklearn.preprocessing import OneHotEncoder

import pandas as pd

# Assuming your test DataFrame is called 'df6'

df6 = test.copy()

# One-hot encode the 'Average Incentive [per month] earned in your pervious company ?'

encoder1 = OneHotEncoder(sparse_output=False)

encoder1.fit(df['Average Incentive [per month] earned in your pervious company ?'].to_numpy().reshape(-1, 1))

encoded_data1 = encoder1.transform(df6[['Average Incentive [per month] earned in your pervious company ?']].to_numpy().reshape(-1, 1))

feature_names_1 = {f'avg_incentive_{i}': i for i in range(encoded_data1.shape[1])}

encoded_df1 = pd.DataFrame(encoded_data1, columns=feature_names_1.keys())

# One-hot encode the 'What was the average ticket size handled at your end in previous role ?'

encoder2 = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoder2.fit(df['What was the average ticket size handled at your end in previous role ?'].to_numpy().reshape(-1, 1))

encoded_data2 = encoder2.transform(df6[['What was the average ticket size handled at your end in previous role ?']].to_numpy().reshape(-1, 1))

feature_names_2 = {f'ticket_size_{i}': i for i in range(encoded_data2.shape[1])}

encoded_df2 = pd.DataFrame(encoded_data2, columns=feature_names_2.keys())

# One-hot encode the 'How many members are there in your family ?'

encoder3 = OneHotEncoder(sparse_output=False)

encoder3.fit(df['How many members are there in your family ?'].to_numpy().reshape(-1, 1))

encoded_data3 = encoder3.transform(df6[['How many members are there in your family ?']].to_numpy().reshape(-1, 1))

feature_names_3 = {f'family_members_{i}': i for i in range(encoded_data3.shape[1])}

encoded_df3 = pd.DataFrame(encoded_data3, columns=feature_names_3.keys())

# One-hot encode the 'Department'

encoder4 = OneHotEncoder(sparse_output=False)

encoder4.fit(df['Department'].to_numpy().reshape(-1, 1))

encoded_data4 = encoder4.transform(df6[['Department']].to_numpy().reshape(-1, 1))

feature_names_4 = {f'department_{i}': i for i in range(encoded_data4.shape[1])}

encoded_df4 = pd.DataFrame(encoded_data4, columns=feature_names_4.keys())

put = [df6.reset_index(drop=True),

encoded_df1.reset_index(drop=True),

encoded_df2.reset_index(drop=True),

encoded_df3.reset_index(drop=True),

encoded_df4.reset_index(drop=True)]

# Concatenate the original DataFrame with the encoded DataFrames

df6 = pd.concat([df6.reset_index(drop=True), encoded_df1.reset_index(drop=True)], axis=1)

df6 = pd.concat([df6.reset_index(drop=True), encoded_df2.reset_index(drop=True)], axis=1)

df6 = pd.concat([df6.reset_index(drop=True), encoded_df3.reset_index(drop=True)], axis=1)

df6 = pd.concat([df6.reset_index(drop=True), encoded_df4.reset_index(drop=True)], axis=1)

test = pd.concat([df6.reset_index(drop=True), test.reset_index(drop=True)], axis=1)

# The feature names are now stored in the following dictionaries:

# feature_names_1, feature_names_2, feature_names_3, feature_names_4

ValueError: Found unknown categories [nan] in column 0 during transform

In [216]:
encoded_df1

,avg_incentive_0,avg_incentive_1,avg_incentive_2,avg_incentive_3,avg_incentive_4
0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...
825,0.0,0.0,1.0,0.0,0.0
826,0.0,0.0,1.0,0.0,0.0
827,0.0,0.0,1.0,0.0,0.0
828,0.0,0.0,1.0,0.0,0.0


In [166]:
test = test.drop(['Average Incentive [per month] earned in your pervious company ?'], axis=1)

In [167]:
test = test.drop(['Department'], axis=1)

In [168]:
test = test.drop(['How many members are there in your family ?'], axis=1)

In [169]:
test = test.drop(['Residential Pincode', 'Branch Pincode'], axis=1)

In [170]:
test['Which Products you are selling in your pervious role ?'] = test['Which Products you are selling in your pervious role ?'].astype(int)

In [171]:
test = test.rename(columns={'Total no of years Experience [before joining Piramal]': 'Total no of years Experience before joining Piramal',
                     'How many are earning family members ? [Other then yourself]2': 'How many are earning family members Other then yourself',
                     'Previous Industry worked with [before joining Piramal]_Banking': 'Previous Industry worked with before joining Piramal_Banking',
                     'Previous Industry worked with [before joining Piramal]_Insurance': 'Previous Industry worked with before joining Piramal_Insurance',
                     'Previous Industry worked with [before joining Piramal]_NBFC':'Previous Industry worked with before joining Piramal_NBFC',
                     'Previous Industry worked with [before joining Piramal]_Non NBFC':'Previous Industry worked with before joining Piramal_Non NBFC',
                     'Previous Industry worked with [before joining Piramal]_Others': 'Previous Industry worked with before joining Piramal_Others',
                     'Previous Industry worked with [before joining Piramal]_others':'Previous Industry worked with before joining Piramal_others'})

In [172]:
pd.set_option('display.max_columns', None)
test

,Total no of years Experience before joining Piramal,How many are earning family members Other then yourself,How many members are dependent on you ?,distance,Previous Industry worked with before joining Piramal_Banking,Previous Industry worked with before joining Piramal_Insurance,Previous Industry worked with before joining Piramal_NBFC,Previous Industry worked with before joining Piramal_Non NBFC,Previous Industry worked with before joining Piramal_Others,Previous Industry worked with before joining Piramal_others,Which Products you are selling in your pervious role ?,education,How did you come to know about the role at Piramal Finance ?,avg_incentive_0,avg_incentive_1,avg_incentive_2,avg_incentive_3,avg_incentive_4,ticket_size_0,ticket_size_1,ticket_size_2,ticket_size_3,ticket_size_4,ticket_size_5,ticket_size_6,ticket_size_7,family_members_0,family_members_1,family_members_2,family_members_3,department_0,department_1,department_2,department_3,department_4,department_5,department_6,department_7
0,10.0,1.0,1,223.930588,0.0,1.0,0.0,0.0,0.0,0.0,10,0,6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,10.0,1.0,1,223.930588,0.0,1.0,0.0,0.0,0.0,0.0,7,0,6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,10.0,1.0,1,223.930588,0.0,1.0,0.0,0.0,0.0,0.0,0,0,6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,5.0,2.0,2,5412.593910,0.0,0.0,1.0,0.0,0.0,0.0,9,0,6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5.0,2.0,2,5412.593910,0.0,0.0,1.0,0.0,0.0,0.0,12,0,6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,10.0,1.0,2,3966.525142,0.0,0.0,1.0,0.0,0.0,0.0,9,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
411,10.0,1.0,2,3966.525142,0.0,0.0,1.0,0.0,0.0,0.0,7,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
412,10.0,1.0,2,3966.525142,0.0,0.0,1.0,0.0,0.0,0.0,12,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
413,10.0,1.0,2,3966.525142,0.0,0.0,1.0,0.0,0.0,0.0,1,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [138]:
test.columns

Index(['Total no of years Experience before joining Piramal',
       'How many are earning family members Other then yourself',
       'How many members are dependent on you ?', 'distance',
       'Previous Industry worked with before joining Piramal_Banking',
       'Previous Industry worked with before joining Piramal_Insurance',
       'Previous Industry worked with before joining Piramal_NBFC',
       'Previous Industry worked with before joining Piramal_Non NBFC',
       'Previous Industry worked with before joining Piramal_Others',
       'Previous Industry worked with before joining Piramal_others',
       'Which Products you are selling in your pervious role ?',
       'How did you come to know about the role at Piramal Finance ?',
       'avg_incentive_0', 'avg_incentive_1', 'avg_incentive_2',
       'avg_incentive_3', 'avg_incentive_4', 'ticket_size_0', 'ticket_size_1',
       'ticket_size_2', 'ticket_size_3', 'ticket_size_4', 'ticket_size_5',
       'ticket_size_6', 'ticket_

In [134]:
X.columns

Index(['Total no of years Experience before joining Piramal',
       'How many are earning family members Other then yourself',
       'How many members are dependent on you ?', 'distance',
       'Previous Industry worked with before joining Piramal_Banking',
       'Previous Industry worked with before joining Piramal_Insurance',
       'Previous Industry worked with before joining Piramal_NBFC',
       'Previous Industry worked with before joining Piramal_Non NBFC',
       'Previous Industry worked with before joining Piramal_Others',
       'Previous Industry worked with before joining Piramal_others',
       'Which Products you are selling in your pervious role ?',
       'How did you come to know about the role at Piramal Finance ?',
       'avg_incentive_0', 'avg_incentive_1', 'avg_incentive_2',
       'avg_incentive_3', 'avg_incentive_4', 'ticket_size_0', 'ticket_size_1',
       'ticket_size_2', 'ticket_size_3', 'ticket_size_4', 'ticket_size_5',
       'ticket_size_6', 'ticket_

In [137]:
test = test[X.columns]

In [139]:
predictions = model.predict(test)

In [145]:
len(predictions)

415

In [142]:
test_data = pd.read_csv('test.csv',encoding='ISO-8859-1')

In [144]:
sub=pd.DataFrame()
sub['CandidateID'],sub['Performance']=test_data['CandidateID'],predictions
sub.to_csv('submission_8.csv', index=False)
sub

ValueError: Length of values (415) does not match length of index (187)